In [1]:
# -*- coding: utf-8 -*-
"""
Created on May 5, 2024
Last modified on May 5, 2024
@Author: Guan-Fu Liu

Description:
To merge the yields from AGB and SNcc
"""
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
sys.path.insert(0, '/Users/liuguanfu/Workspace/SAS-21/targets/MRK1216/spex/jupyter/IMF/chemevoimf')
import utils
import h5py
import shutil

In [2]:
out_dir = './AGB+SNcc/'

yields_sets = { }

yields_sets['agb_and_massive_stars_K10_N13'] = [
    ('./AGB/N13-0.0000-AGB-total.csv', './SNcc/N13-0.0000-SNcc-total.csv', 'Z=0.0000'), 
    ('./AGB/K10-0.0001-AGB-total.csv', './SNcc/N13-0.0010-SNcc-total.csv', 'Z=0.0001'),
    ('./AGB/K10-0.0040-AGB-total.csv', './SNcc/N13-0.0040-SNcc-total.csv', 'Z=0.0040'),
    ('./AGB/K10-0.0080-AGB-total.csv', './SNcc/N13-0.0080-SNcc-total.csv', 'Z=0.0080'),
    ('./AGB/K10-0.0200-AGB-total.csv', './SNcc/N13-0.0200-SNcc-total.csv', 'Z=0.0200'),
    ('./AGB/K10-0.0200-AGB-total.csv', './SNcc/N13-0.0500-SNcc-total.csv', 'Z=0.0500'),
    ]

yields_sets['agb_and_massive_stars_K10_N13_S16N20'] = [
    ('./AGB/N13-0.0000-AGB-total.csv', './SNcc/N13-0.0000-SNcc-total.csv', 'Z=0.0000'), 
    ('./AGB/K10-0.0001-AGB-total.csv', './SNcc/N13-0.0010-SNcc-total.csv', 'Z=0.0001'),
    ('./AGB/K10-0.0040-AGB-total.csv', './SNcc/N13-0.0040-SNcc-total.csv', 'Z=0.0040'),
    ('./AGB/K10-0.0080-AGB-total.csv', './SNcc/N13-0.0080-SNcc-total.csv', 'Z=0.0080'),
    ('./AGB/K10-0.0200-AGB-total.csv', './SNcc/N13-0.0500-SNcc-total.csv', 'Z=0.0500'),
    ('./AGB/K10-0.0200-AGB-total.csv', './SNcc/S16N20-0.0200-SNcc-total.csv', 'Z=0.0200'),
    ]

yields_sets['agb_and_massive_stars_K10_N13_S16W18'] = [
    ('./AGB/N13-0.0000-AGB-total.csv', './SNcc/N13-0.0000-SNcc-total.csv', 'Z=0.0000'), 
    ('./AGB/K10-0.0001-AGB-total.csv', './SNcc/N13-0.0010-SNcc-total.csv', 'Z=0.0001'),
    ('./AGB/K10-0.0040-AGB-total.csv', './SNcc/N13-0.0040-SNcc-total.csv', 'Z=0.0040'),
    ('./AGB/K10-0.0080-AGB-total.csv', './SNcc/N13-0.0080-SNcc-total.csv', 'Z=0.0080'),
    ('./AGB/K10-0.0200-AGB-total.csv', './SNcc/N13-0.0500-SNcc-total.csv', 'Z=0.0500'),
    ('./AGB/K10-0.0200-AGB-total.csv', './SNcc/S16W18-0.0200-SNcc-total.csv', 'Z=0.0200'),
    ]

comments = { }
comments['agb_and_massive_stars_K10_N13'] = 'AGB and SNcc yields from Karakas (2010) and Nomoto et al. (2013)'
comments['agb_and_massive_stars_K10_N13_S16N20'] = \
    'AGB and SNcc yields from Karakas (2010) and Nomoto et al. (2013) and N20 model (solar metallicity) of Sukhbold et al. (2016)'
comments['agb_and_massive_stars_K10_N13_S16W18'] = \
    'AGB and SNcc yields from Karakas (2010) and Nomoto et al. (2013) and W18 model (solar metallicity) of Sukhbold et al. (2018)'

for key in yields_sets:
    yields_set = pd.DataFrame(yields_sets[key], columns=['AGB', 'SNcc', 'Label'])
    yields_sets[key] = yields_set

In [3]:
for key in yields_sets.keys():
    if not os.path.exists(out_dir+key):
        os.makedirs(out_dir+key)
    for row1 in yields_sets[key].iterrows():
        AGB = pd.read_csv(row1[1]['AGB'])
        SNcc = pd.read_csv(row1[1]['SNcc'])
        df1 = pd.merge(AGB, SNcc, on='M', how='inner')
        df1.set_index('M', inplace=True)
        df1.to_csv("%s%s/%s.csv"%(out_dir, key, row1[1]['Label']))
        # Mass grids where the yields are to be extra or interpolated
        columns = ["%0.2e" % a for a in np.logspace(np.log10(0.08), np.log10(150), 300)]
        # Empty dataframe with the same index as df1
        df2 = pd.DataFrame(np.zeros((len(df1.index), len(columns))), index=df1.index, columns=columns)
        for i, row2 in df1.iterrows():
            x1 = row2.index.to_numpy().astype(float)  # x1 is the initial mass from original yields
            y1 = row2.to_numpy().astype(float)  # y1 is the remnant mass from original yields
            df2.loc[row2.name, :] = [utils.extra_interpolate_yields(x1, y1, row2.name, float(col)) for col in columns]
        df2.to_csv("%s%s/%s_interpolated.csv"%(out_dir, key, row1[1]['Label']))
    with h5py.File("%s%s/yields1.h5"%(out_dir, key), 'w') as f:
        f.attrs['Comments'] = comments[key]
        for label in yields_sets[key]['Label']:
            # Create a group for each initial metallicity
            f.create_group(label)
            # hdf5 does not support string with object type
            # "|S" will find the maximum length of the string in the selected column
            df1 = pd.read_csv("%s%s/%s.csv"%(out_dir, key, label))
            df1['M'] = df1['M'].astype('|S')
            data = df1.to_records(index=False)
            dtype = df1.to_records(index=False).dtype
            f[label].attrs['Z'] = "%s" % label[2:]
            f[label].attrs['MassUnit'] = 'Msun'
            f[label].create_dataset('Original', data=data, dtype=dtype)

            df2 = pd.read_csv("%s%s/%s_interpolated.csv"%(out_dir, key, label))            
            df2['M'] = df2['M'].astype('|S')
            data = df2.to_records(index=False)
            dtype = df2.to_records(index=False).dtype
            f[label].create_dataset('Interpolated', data=df2.to_records(), dtype=dtype)
    
    # yields2.h5 is the other kind of hdf5 file by using to_hdf
    # but the content is the same as yields1.h5
    with h5py.File("%s%s/yields2.h5"%(out_dir, key), 'w') as f:
        f.attrs['Comments'] = comments[key]
    for label in yields_sets[key]['Label']:
        # Create a group for each initial metallicity
        df1 = pd.read_csv("%s%s/%s.csv"%(out_dir, key, label), index_col=0)
        df1.to_hdf("%s%s/yields2.h5"%(out_dir, key), key='/%s/Original' % label.replace("=", "_").replace(".", "_"), mode='a')
        df2 = pd.read_csv("%s%s/%s_interpolated.csv"%(out_dir, key, label), index_col=0)
        df2.to_hdf("%s%s/yields2.h5"%(out_dir, key), key='/%s/Interpolated' % label.replace("=", "_").replace(".", "_"), mode='a')
    # Add attributes to the datasets
    with h5py.File("%s%s/yields2.h5"%(out_dir, key), 'a') as f:
        for label in yields_sets[key]['Label']:
            f[label.replace("=", "_").replace(".", "_")].attrs['Z'] = "%s" % label[2:]
            f[label.replace("=", "_").replace(".", "_")].attrs['MassUnit'] = 'Msun'

In [4]:
# Copy the NuPyCEE directory to the inputs directory
if os.path.exists('../inputs/K10_N13_S16'):
    shutil.rmtree('../inputs/K10_N13_S16')
shutil.copytree('./AGB+SNcc', '../inputs/K10_N13_S16')

'../inputs/K10_N13_S16'